In [1]:
#Based on https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/2_BasicModels/linear_regression.ipynb
import tensorflow as tf
import numpy
import csv
rng = numpy.random

In [2]:
# Parameters
learning_rate = 0.01
training_epochs = 300
display_step = 50
#We currently turn this off because it makes the model more difficult to apply to real data, and 
#because it doesn't appear to be necessary for convergence.
DO_NORMALIZE = False

In [3]:
#Training data
csv = numpy.genfromtxt("solves.csv", skip_header=1, delimiter=",")
train_difficulty = csv[:, 0]
train_features = csv[:, 1:]

In [4]:
#number of training samples
n_samples = train_difficulty.shape[0]

#number of features
feature_length = train_features.shape[1]

In [ ]:
#Normalize all of the columns to be between -0.5 and 0.5 with a smooth distribution
if DO_NORMALIZE:
    #TODO: we need to save the maxVal so other folks in the model can normalize appropriately
    #TODO: figure out a better way to normalize these; right now many of them are squishing at -0.5 since their
    #distributions are often very left-skewed
    print("Normalizing")
    for colNum in xrange(0, feature_length):
        col = train_features[:,colNum]
        maxVal = max(col)
        if maxVal == 0:
            maxVal = 1
        for index in xrange(0, len(col)):
            col[index] = (col[index] - (maxVal / 2)) / maxVal
else:
    print("Skipping normalizing")

In [5]:

# tf Graph Input
Difficulty = tf.placeholder("float")
Features = tf.placeholder("float", shape=(feature_length))

# Set model weights
W = tf.Variable(tf.random_normal([feature_length], stddev=0.25), name="weight")
b = tf.Variable(0.0, name="bias")

In [6]:
# Construct a linear model
pred = tf.add(tf.mul(Features, W), b)

In [7]:
#TODO: calculate R2 and output it over time
# Mean squared error
with tf.name_scope("cost") as scope:
    cost = tf.reduce_sum(tf.pow(pred-Difficulty, 2))/(2*n_samples)
    cost_summ = tf.scalar_summary("cost", cost)
# Gradient descent
with tf.name_scope("optimize") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [8]:
#Summary ops to collect data
#The use of tensorboard is adapted from the example on https://www.tensorflow.org/versions/r0.7/how_tos/summaries_and_tensorboard/index.html
w_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
difficulty_hist = tf.histogram_summary("difficulty", Difficulty)

In [9]:
# Initializing the variables
init = tf.initialize_all_variables()

In [10]:
# Launch the graph

#TODO: do STOCHASTIC gradient descent
with tf.Session() as sess:
    
    merged = tf.merge_all_summaries()
    #TODO: figure out how to get these to overwrite existing logs (or at least be separate)
    writer = tf.train.SummaryWriter("tmp/linear_logs", sess.graph_def)
    
    sess.run(init)

    # Fit all training data
    for epoch in range(training_epochs):
        for (features, difficulty) in zip(train_features, train_difficulty):
            sess.run(optimizer, feed_dict={Features: features, Difficulty: difficulty})

        #Display logs per epoch step
        if (epoch+1) % display_step == 0:
            result = sess.run([merged,cost], feed_dict={Features: features, Difficulty: difficulty})
            summary_str = result[0]
            c = result[1]
            writer.add_summary(summary_str, epoch)
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "W=", sess.run(W), "b=", sess.run(b)

    print "Optimization Finished!"
    training_cost = sess.run(cost, feed_dict={Features: features, Difficulty: difficulty})
    print "Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n'


Epoch: 0050 cost= 0.006754194 W= [-0.12339126 -0.46053571 -0.04670807 -0.19390152  0.21711847  0.2669456
  0.02571134  0.1887003   0.1517283   0.02213729  0.00227103  0.20010909
 -0.06192914 -0.25108793  0.24930021  0.21471448 -0.27096069 -0.11343871
 -0.00409989 -0.294236    0.26401386  0.21977346 -0.11212447 -0.03615963
  0.15228313  0.32542157  0.49856156  0.0852884  -0.4309364   0.08565165
  0.0243285   0.2478058  -0.01933379 -0.09320458  0.15058093 -0.64584363
  0.12525648  0.21167934 -0.21437828 -0.37254348  0.33566993 -0.40179864
 -0.13289675 -0.57010233  0.03365078 -0.61660826 -0.24922524  0.32886368
  0.4356609  -0.4295609  -0.04322536 -0.24863817  0.0103194   0.18855175
  0.0094152   0.26110479  0.00919067 -0.28044772  0.00787557  0.00747317
  0.00506957 -0.29148462  0.00531697  0.19599319  0.00742508  0.3150008
  0.00904437  0.22432071  0.39174289  0.1839904   0.10398999  0.34787416
  0.07707904  0.00961297 -0.25711244 -0.35950601  0.0506169   0.38394645
  0.16658008  0.3309

In [ ]:
#TODO: output the model with bias, cost, and weights all zipped up with their names